# 🧠 Introduction to Agentic Workflows in Business

In today’s business environment, Artificial Intelligence (AI) is evolving beyond single-prompt systems.  
Rather than responding to one-time queries, **Agentic AI** allows models to **think, plan, and act** across multiple steps — much like a human assistant that reasons through problems before giving a final answer.

## 🤖 What Is Agentic AI?

Traditional use of large language models (LLMs) often looks like this:

> “Write an essay about topic X.”

The model outputs everything in one pass — from the first word to the last — without re-thinking or revising.  
While this can work, it’s limited. Humans don’t write or make decisions this way either — we **draft, reflect, research, and revise**.

Agentic workflows, on the other hand, break a complex task into **smaller, iterative steps** such as:

1. Planning or outlining the goal  
2. Searching for information or collecting data  
3. Drafting or executing the task  
4. Reflecting on what needs improvement  
5. Revising or requesting human review before finalizing

This approach mirrors how experts work in the real world — continuously refining until the outcome meets a clear objective.

---

## 💼 Why Agentic Workflows Matter in Business

In business, agentic workflows unlock **autonomy, accuracy, and efficiency**.  
They enable AI systems to act like *digital assistants* that can **research, compare, evaluate, and recommend** before making a decision.

For example:

> **Task:** Find the best supplier for laboratory instruments under \$10,000.

An agentic system could:
1. Search multiple supplier sites and extract product data.  
2. Compare features, prices, and reviews.  
3. Rank the top options using defined business rules.  
4. Ask for human approval before generating a purchase recommendation.

Instead of giving one-off results, the system **iterates**, **checks its work**, and even **collaborates with a human** — leading to better business decisions.

---

## 🎯 What You’ll Learn in This Tutorial

In this tutorial, we’ll create a **simple agentic workflow** for a common business task:  
> Asking an AI agent to find and recommend products or services to buy based on user needs.

You’ll learn how to:

1. **Decompose** a business task into smaller reasoning steps.  
2. **Build** each step using LLM-powered functions.  
3. **Connect** these steps into an iterative, goal-driven workflow.  
4. Optionally, **add a human-in-the-loop** review process for better reliability.

---

## 🧩 The Goal

By the end of this tutorial, you’ll move from a simple prompt like:

> “Find me a good product to buy.”

to a structured **multi-step agent** that can **plan, search, analyze, and recommend** —  
laying the foundation for more advanced business agents such as:

- Product recommendation assistants  
- Market research bots  
- Customer service agents  
- Supplier evaluation agents

---

> 💡 *Inspired by the DeepLearning.AI course on Agentic AI, this tutorial adapts those core ideas into a practical business workflow that you can build and extend inside Google Colab.*


# 🤖 Agentic Customer Service Agent (Multi-tool + Reflection)

In this tutorial, you’ll learn how to build a **multi-tool agentic workflow** that acts like a smart customer service assistant.  
The agent can **think, plan, act, and reflect** to answer customer requests about products in an eyewear shop.

We'll go step by step:
1. Set up the environment and database  
2. Build each tool (functions)  
3. Combine everything into an agent  
4. Reflect and summarize results  
5. Review key concepts with a short quiz


In [2]:
# ==========================================================
# 📦 SETUP
# ==========================================================

!pip install google-generativeai tinydb -q

import google.generativeai as genai
from tinydb import TinyDB, Query
import json, re, time

# ---- CONFIG ----
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')  # if running in DLAI env
genai.configure(api_key=GOOGLE_API_KEY)


## 🗃️ Step 1: Create a Simple Product Database

We’ll use **TinyDB** to simulate an in-memory product database.  
This database includes three example products with fields such as `name`, `price`, `stock`, and `description`.


In [3]:
# ---- DATABASE ----
db = TinyDB("products.json")
db.truncate()
db.insert_multiple([
    {
        "name": "Smart Glasses X1",
        "price": 120,
        "stock": 8,
        "description": "AR-powered glasses with touch control and Bluetooth audio."
    },
    {
        "name": "Reading Glasses Classic",
        "price": 40,
        "stock": 15,
        "description": "Lightweight reading glasses for everyday use."
    },
    {
        "name": "Sun Glasses Retro",
        "price": 60,
        "stock": 0,
        "description": "UV-protected polarized retro sunglasses."
    }
])


[1, 2, 3]

In [4]:
# ---- CUSTOMER DATABASE ----
customers_db = TinyDB("customers.json")
customers_db.truncate()
customers_db.insert_multiple([
    {
        "id": 1,
        "name": "John Doe",
        "credits": 300
    },
    {
        "id": 2,
        "name": "Sarah Lee",
        "credits": 150
    },
    {
        "id": 3,
        "name": "David Kim",
        "credits": 500
    }
])

print("✅ Customers database created.")


✅ Customers database created.


In [9]:
# ---- INVOICES DATABASE ----
invoices_db = TinyDB("invoices.json")
invoices_db.truncate()

invoices_db.insert_multiple([
    {
        "invoice_id": 1,
        "customer_id": 1,
        "product_name": "Smart Glasses X1",
        "price": 120,
        "purchase_date": "2025-01-15",
        "warranty_months": 12
    },
    {
        "invoice_id": 2,
        "customer_id": 2,
        "product_name": "Reading Glasses Classic",
        "price": 40,
        "purchase_date": "2025-02-10",
        "warranty_months": 6
    },
    {
        "invoice_id": 3,
        "customer_id": 1,
        "product_name": "Sun Glasses Retro",
        "price": 60,
        "purchase_date": "2024-11-20",
        "warranty_months": 3
    }
])

print("✅ Invoices database created with 3 sample records.")


✅ Invoices database created with 3 sample records.


In [8]:
from tinydb import Query
Customer = Query()
print(customers_db.search(Customer.name == "John Doe"))


[{'id': 1, 'name': 'John Doe', 'credits': 300}]


In [10]:
from tinydb import Query
Invoice = Query()
print(invoices_db.search(Invoice.customer_id == 1))


[{'invoice_id': 1, 'customer_id': 1, 'product_name': 'Smart Glasses X1', 'price': 120, 'purchase_date': '2025-01-15', 'warranty_months': 12}, {'invoice_id': 3, 'customer_id': 1, 'product_name': 'Sun Glasses Retro', 'price': 60, 'purchase_date': '2024-11-20', 'warranty_months': 3}]


## 🧩 Step 2: Product Description Tool (`get_product_description`)

This tool retrieves a product’s **description** from the database.

**Concept:**  
The agent calls this when a user asks for product details, like:  
> “Tell me about Smart Glasses X1.”


In [19]:
def get_product_description(product_name: str):
    Product = Query()
    result = db.search(Product.name == product_name)
    if result:
        return result[0]["description"]
    return f"❌ Sorry, no description found for {product_name}."

# ✅ Test
print(get_product_description("Smart Glasses X1"))


AR-powered glasses with touch control and Bluetooth audio.


## 🔍 Step 3: Stock Checker Tool (`check_stock`)

This tool checks how many units of a product are available.  
It returns a friendly message about whether the product is in stock or out of stock.


In [20]:
def check_stock(product_name: str):
    Product = Query()
    result = db.search(Product.name == product_name)
    if result:
        stock = result[0]["stock"]
        if stock > 0:
            return f"✅ {product_name} is in stock ({stock} units available)."
        else:
            return f"⚠️ {product_name} is currently out of stock."
    return f"❌ Product '{product_name}' not found."

# ✅ Test
print(check_stock("Reading Glasses Classic"))
print(check_stock("Sun Glasses Retro"))


✅ Reading Glasses Classic is in stock (15 units available).
⚠️ Sun Glasses Retro is currently out of stock.


## 💰 Step 4: Product Recommendation Tool (`recommend_product`)

This tool suggests products under a given **budget**.  
The agent uses it when a user asks:
> “Do you have any glasses under 100 dollars?”


In [21]:
def recommend_product(budget: float):
    Product = Query()
    results = db.search(Product.price <= budget)
    if not results:
        return f"❌ No products found under ${budget}."
    suggestions = [f"- {r['name']} (${r['price']})" for r in results]
    return "Here are some products within your budget:\n" + "\n".join(suggestions)

# ✅ Test
print(recommend_product(100))


Here are some products within your budget:
- Reading Glasses Classic ($40)
- Sun Glasses Retro ($60)


**Check Customer Credits**





In [11]:
# ✅ NEW TOOL: Check Customer Credits
def check_customer_credits(customer_name: str):
    Customer = Query()
    result = customers_db.search(Customer.name == customer_name)

    if not result:
        return f"❌ Customer '{customer_name}' not found."

    credits = result[0]["credits"]
    return f"💳 {customer_name} has {credits} store credits available."


In [12]:
print(check_customer_credits("John Doe"))
print(check_customer_credits("Sarah Lee"))
print(check_customer_credits("Unknown Person"))


💳 John Doe has 300 store credits available.
💳 Sarah Lee has 150 store credits available.
❌ Customer 'Unknown Person' not found.


**Find Customer**

In [13]:
# ✅ Tool: Find customer invoices
def find_customer_invoices(customer_name: str):
    # find customer ID
    Customer = Query()
    customer = customers_db.search(Customer.name == customer_name)
    if not customer:
        return f"❌ Customer '{customer_name}' not found."

    customer_id = customer[0]["id"]

    # find invoices
    Invoice = Query()
    records = invoices_db.search(Invoice.customer_id == customer_id)

    if not records:
        return f"🧾 No invoices found for {customer_name}."

    message = f"🧾 Purchase history for {customer_name}:\n"
    for inv in records:
        message += (
            f"- Invoice #{inv['invoice_id']} | {inv['product_name']} | "
            f"${inv['price']} | Purchased: {inv['purchase_date']} | "
            f"Warranty: {inv['warranty_months']} months\n"
        )
    return message.strip()


In [14]:
print(find_customer_invoices("John Doe"))


🧾 Purchase history for John Doe:
- Invoice #1 | Smart Glasses X1 | $120 | Purchased: 2025-01-15 | Warranty: 12 months
- Invoice #3 | Sun Glasses Retro | $60 | Purchased: 2024-11-20 | Warranty: 3 months


**Check Warranty**

In [17]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# ✅ Tool: Check warranty by invoice ID
def check_warranty(invoice_id: int):
    Invoice = Query()
    record = invoices_db.search(Invoice.invoice_id == invoice_id)

    if not record:
        return f"❌ Invoice #{invoice_id} not found."

    invoice = record[0]
    purchase_date = datetime.strptime(invoice["purchase_date"], "%Y-%m-%d")
    warranty_months = invoice["warranty_months"]
    warranty_end = purchase_date + relativedelta(months=warranty_months)

    today = datetime.today()

    if today <= warranty_end:
        return f"✅ Invoice #{invoice_id} is still under warranty until {warranty_end.date()}."
    else:
        return f"⚠️ Warranty expired on {warranty_end.date()} for Invoice #{invoice_id}."

In [18]:
print(check_warranty(1))
print(check_warranty(3))
print(check_warranty(999))


✅ Invoice #1 is still under warranty until 2026-01-15.
⚠️ Warranty expired on 2025-02-20 for Invoice #3.
❌ Invoice #999 not found.


## ⚙️ Step 5: JSON Parser Utility (`parse_json_response`)

This helper function extracts valid JSON from model outputs.  
It ensures the agent can interpret tool-call instructions correctly, even if the model adds formatting like ```json.


In [22]:
def parse_json_response(raw_text):
    clean = re.sub(r"```json|```", "", raw_text).strip()
    match = re.search(r"\{.*\}", clean, re.DOTALL)
    if not match:
        raise ValueError(f"No JSON found in model output:\n{raw_text[:200]}")
    return json.loads(match.group(0))


## 🧠 Step 6: Building the Agent (`agentic_customer_service`)

This is the main **agentic workflow**.  
It can:
1. Decide which tool(s) to use  
2. Execute them step by step  
3. Reflect and summarize in a customer-friendly way


In [30]:
def agentic_customer_service(user_prompt: str, max_steps: int = 3):
    """
    Multi-tool Agentic workflow with reflection.
    - LLM decides which tools to call (1 or more)
    - Executes sequentially, stores results
    - Reflects to produce final natural answer
    """
    model = genai.GenerativeModel("gemini-2.5-flash")

    tools_description = """
    You have access to these tools:
    1. get_product_description(product_name: str)
    2. check_stock(product_name: str)
    3. recommend_product(budget: float)
    4. check_customer_credits(customer_name: str)
    5. find_customer_invoices(customer_name: str)
    6. check_warranty(invoice_id: int)
    7. buy_product(customer_name: str, product_name: str)

    Each step, return JSON:
    {"actions": [
        {"tool": "tool_name", "args": {...}},
        ...
    ]}
    You may call one or more tools depending on user need.
    """

    history = []
    print(f"🗣️ User: {user_prompt}\n")

    for step in range(1, max_steps + 1):
        step_prompt = f"""
        You are an AI assistant for an eyewear shop.
        {tools_description}

        Conversation so far:
        {json.dumps(history, indent=2)}

        User request: {user_prompt}

        Decide what tools to use next (if any), or return "done" if ready to answer.
        Output strictly in JSON.
        """

        response = model.generate_content(step_prompt)
        text = response.text.strip()
        print(f"🔹 Step {step} Decision:\n{text}\n")

        # --- FIX: Handle 'done' or non-JSON output gracefully ---
        if text.lower() == "done":
            print("✅ No more actions. Proceeding to reflection.\n")
            break

        # Try parsing JSON
        try:
            decision_data = parse_json_response(text)
            actions = decision_data.get("actions", [])
        except Exception as e:
            print(f"⚠️ Parsing failed: {e}")
            break

        if not actions:
            print("✅ No actions found. Proceeding to reflection.\n")
            break

        # Execute each tool and log results
        for action in actions:
            tool = action.get("tool")
            args = action.get("args", {})
            print(f"🔧 Executing: {tool} {args}")

            try:
                if tool == "get_product_description":
                    result = get_product_description(**args)
                elif tool == "check_stock":
                    result = check_stock(**args)
                elif tool == "recommend_product":
                    result = recommend_product(**args)
                elif tool == "check_customer_credits":
                    result = check_customer_credits(**args)
                elif tool == "find_customer_invoices":
                    result = find_customer_invoices(**args)
                elif tool == "check_warranty":
                    result = check_warranty(**args)
                elif tool == "buy_product":
                    result = buy_product(**args)
                else:
                    result = f"Unknown tool '{tool}'"
            except Exception as e:
                result = f"Error while executing {tool}: {e}"

            history.append({"tool": tool, "args": args, "result": result})
            print(f"📦 Result: {result}\n")

        time.sleep(1)

    # ---- Reflection Phase ----
    reflection_prompt = f"""
    The user asked: {user_prompt}
    You have this context from tools:
    {json.dumps(history, indent=2)}

    Write a final, polite, customer-friendly summary.
    Don't mention tools or steps. Just the helpful final answer.
    """

    final = model.generate_content(reflection_prompt)
    print("💬 Final Response:\n")
    return final.text


## 🪞 Step 7: Run the Agent (Full Workflow)

Now, let’s test everything together.  
The agent will plan, act, and reflect like a real assistant.


In [32]:
print(agentic_customer_service("My name is John Doe, i want to buy Smart Glasses X1"))


🗣️ User: My name is John Doe, i want to buy Smart Glasses X1

🔹 Step 1 Decision:
```json
{
  "actions": [
    {
      "tool": "check_stock",
      "args": {
        "product_name": "Smart Glasses X1"
      }
    }
  ]
}
```

🔧 Executing: check_stock {'product_name': 'Smart Glasses X1'}
📦 Result: ✅ Smart Glasses X1 is in stock (7 units available).

🔹 Step 2 Decision:
```json
{
  "actions": [
    {
      "tool": "buy_product",
      "args": {
        "customer_name": "John Doe",
        "product_name": "Smart Glasses X1"
      }
    }
  ]
}
```

🔧 Executing: buy_product {'customer_name': 'John Doe', 'product_name': 'Smart Glasses X1'}
📦 Result: {'status': 'success', 'message': 'Purchase completed successfully.', 'invoice_id': 5, 'product': 'Smart Glasses X1', 'price': 120, 'remaining_credits': 60}

🔹 Step 3 Decision:
"done"

⚠️ Parsing failed: No JSON found in model output:
"done"
💬 Final Response:

Hello John Doe,

Your purchase of the Smart Glasses X1 has been successfully completed!



# ✅ Reflection Quiz

Answer the following questions to review your understanding:

1. **Planning:**  
   How does the agent decide which tool(s) to use for a specific request?  
   (Hint: Look at the JSON output.)

2. **Action:**  
   Why is the JSON structure important for executing the correct tool?

3. **Reflection:**  
   What does the reflection phase contribute to the final answer?

4. **Extension:**  
   If you wanted the agent to also tell you *the cheapest* product,  
   which new tool would you add?

🧩 *Discuss these questions or write your answers in a new text cell.*


# 🧩 Step 8: Student Exercise — Create a New, Unique Tool

Now it’s your turn to extend the agent’s intelligence!

The agent currently has three tools:
1. `recommend_product(budget)` → finds products within a budget  
2. `check_stock(product_name)` → checks availability  
3. `get_product_description(product_name)` → gives product info  

Your task is to **create a new tool that provides a capability not covered by these.**

---

## 🧠 Design Challenge

Think about what *else* a customer might ask that cannot be answered by the existing tools.

Here are some example ideas:
| New Tool Name | What It Does | Example User Question |
|----------------|---------------|------------------------|
| `compare_products(product1, product2)` | Compares two products by price, stock, or features | “Compare Smart Glasses X1 and Reading Glasses Classic.” |
| `get_average_price()` | Calculates and returns the average price of all products | “What’s the average price of your glasses?” |
| `list_available_products()` | Returns only products currently in stock | “Which glasses are currently available?” |
| `get_highest_stock_item()` | Finds the product with the largest stock | “Which product do you have the most of?” |
| `estimate_total_value()` | Calculates total store value (price × stock for all items) | “How much is your total stock worth?” |

Choose one of these, or come up with your own!  
Your tool should add **new reasoning ability** to the agent — something it could *never* do before.

---

## 🧱 Step 8A: Define Your Tool


In [ ]:
# 🧩 Example: list_available_products()
# You can change this idea or make your own unique function!

def list_available_products():
    """
    Lists all products that are currently in stock.
    """
    products = db.all()
    available = [p for p in products if p["stock"] > 0]

    if not available:
        return "⚠️ No products are available at the moment."

    message = "🟢 Available Products:\n"
    for p in available:
        message += f"- {p['name']} (${p['price']}) — {p['stock']} in stock\n"
    return message.strip()

# ✅ Test your tool
print(list_available_products())


🟢 Available Products:
- Smart Glasses X1 ($120) — 8 in stock
- Reading Glasses Classic ($40) — 15 in stock


## 🧠 Step 8C: Test Your Extended Agent

Now test your agent with a question that uses your new capability.

Try something like:
> “Can you list all products that are available right now?”


**Advanced/ Complex Tools**

In [27]:
from datetime import datetime

# ✅ New Tool: Buy Product
def buy_product(customer_name: str, product_name: str):
    # --- Check customer exists ---
    Customer = Query()
    customer = customers_db.search(Customer.name == customer_name)
    if not customer:
        return {
            "status": "error",
            "message": f"Customer '{customer_name}' not found."
        }
    customer = customer[0]

    # --- Check product exists ---
    Product = Query()
    product = db.search(Product.name == product_name)
    if not product:
        return {
            "status": "error",
            "message": f"Product '{product_name}' not found."
        }
    product = product[0]

    # --- Check stock ---
    if product["stock"] <= 0:
        return {
            "status": "error",
            "message": f"Product '{product_name}' is out of stock."
        }

    # --- Check credits ---
    if customer["credits"] < product["price"]:
        return {
            "status": "error",
            "message": f"Insufficient credits. {customer_name} has {customer['credits']} credits, price is {product['price']}."
        }

    # --- Deduct credits & update customer ---
    new_credits = customer["credits"] - product["price"]
    customers_db.update({"credits": new_credits}, Customer.id == customer["id"])

    # --- Update stock ---
    new_stock = product["stock"] - 1
    db.update({"stock": new_stock}, Product.name == product_name)

    # --- Create invoice ---
    invoice_id = len(invoices_db) + 1
    purchase_date = datetime.today().strftime("%Y-%m-%d")
    warranty_months = 12
    invoices_db.insert({
        "invoice_id": invoice_id,
        "customer_id": customer["id"],
        "product_name": product_name,
        "price": product["price"],
        "purchase_date": purchase_date,
        "warranty_months": warranty_months
    })

    # --- Return success message ---
    return {
        "status": "success",
        "message": "Purchase completed successfully.",
        "invoice_id": invoice_id,
        "product": product_name,
        "price": product["price"],
        "remaining_credits": new_credits
    }


In [28]:
print(buy_product("John Doe", "Smart Glasses X1"))

{'status': 'success', 'message': 'Purchase completed successfully.', 'invoice_id': 4, 'product': 'Smart Glasses X1', 'price': 120, 'remaining_credits': 180}
